# Classification
## MSDS 7349 - Section 401
## Project 2

[Data Science @ Southern Methodist University](https://datascience.smu.edu/)

# Table of Contents
* [Team Members](#Team-Members)
* [Data Preparation](#Data-Preparation)
* [Dataset Description](#Dataset-Description)
* [Evaluation Metrics Description](#Evaluation-Metrics-Description)
* [Training and Testing Splits method](#Training-Testing)
* [Classification Task](#Classification-Task)
* [Regression Task](#Regression-Task)
* [Methods of Evaluation Results](#Methods-of-Evaluation)
* [Advantages of each model](#Advantages-of-each-model)
* [Relevant Attributes](#Relevant-Attributes)
* [Deployment](#Deployment)
* [Exceptional Work](#Exceptional-Work)
* [References](#References)


# <a name="Team-Members"></a>Team Members
* [Jostein Barry-Straume](https://github.com/josteinstraume)
* [Kevin Cannon](https://github.com/kcannon2)
* [Ernesto Carrera Ruvalcaba](https://github.com/ecarrerasmu)
* [Adam Tschannen](https://github.com/adamtschannen)

# <a name="Data-Preparation"></a>Data Preparation

[10 points] Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

> The credit default data set is from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients#). It has 30,000 records with 24 attributes. The records reflect the payment data from an important Taiwanese bank during October, 2005 (Yeh & Lien 2475).

> The purpose of the data set was to tackle the issue of forecasting the probability of default. In particular, the researchers of this data set sought to determine if the "estimated probability of default produced from data mining methods... (could) represent the "real" probability of default" (Yeh & Lien 2473).

> The dataset needs some pre-processing before it is ready for classification and regression analysis.

> To start, missing or incorrect values for education level, marital status, and repayment status are imputed.

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sns
import statistics as st
import csv as csv
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from __future__ import print_function
from sklearn.preprocessing import StandardScaler


#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

filepath = "/data/credit-defaults.xls"

# Relative local path to avoid changing filepath constantly
credit = pd.read_excel('../data/credit-defaults.xls', header=1, skiprows=0)


# Rename column(s)
credit = credit.rename(columns={'default payment next month': 'default_next_m', 'PAY_0': 'PAY_1'})


In [85]:
#CREATING NEW VARIABLE TO IMPUTE THE VALUES TO 4, THAT REPRESENTS OTHER
credit['EDUCATION_INP']=credit['EDUCATION']

credit.loc[credit['EDUCATION'] > 4, 'EDUCATION_INP'] = 4
credit.loc[credit['EDUCATION'] == 0, 'EDUCATION_INP'] = 4

In [86]:
####FOR MARRIAGE
#CREATING NEW VARIABLE TO IMPUTE THE VALUES TO 2, THAT REPRESETNS SINGLE, since the value of 0 is not defined in
#the data dictionary
credit['MARRIAGE_INP']=credit['MARRIAGE']


credit.loc[credit['MARRIAGE'] == 0, 'MARRIAGE_INP'] = 2


In [87]:
########### CREATING A FUNCTION TO GROUP VARIABLES
def group_biv_v1(var_name,RangeName,CutOff,NumBins,BinLabel,var_x,var_y,title1,title2):
#def bivariate_v1(NumBins): 
    #Creating the BINS
    credit[RangeName]=pd.cut(var_name,CutOff,NumBins,labels=BinLabel)
    #Grouping by the BINS variable
    default_g=credit.groupby(by=[RangeName])
    #% of POPULATION
    bins_percentage=default_g[RangeName].count()/credit[RangeName].count()
    
    #### creating DUMMY VARIABLES
    
    
    
    

#group_biv_v1(credit.PAY_1,'PAY_1_range',[-2.1,0,1,10],3,['Pay duly','delay 1 month ','delay > 1'],'Percentage','PAY_1_bins','% of population PAY_1','Default PAY_1 ')


In [88]:
############## IMPUTE VALUES FOR THE VARIABLES OF REPAYMENT STATUS (PAY0, PAY1,   PAYN)
group_biv_v1(credit.PAY_1,'PAY_1_range',[-2.1,0,1,10],3,['Pay duly','delay 1 month ','delay > 1'],'Percentage','PAY_1_bins','% of population PAY_1','Default PAY_1 ')

group_biv_v1(credit.PAY_2,'PAY_2_range',[-2.1,1,10],2,['Pay duly','delay > 1'],'Percentage','PAY_2_bins','% of population PAY_2','Default PAY_2 ')

group_biv_v1(credit.PAY_3,'PAY_3_range',[-2.1,1,10],2,['Pay duly','delay > 1'],'Percentage','PAY_3_bins','% of population PAY_3','Default PAY_3 ')

group_biv_v1(credit.PAY_4,'PAY_4_range',[-2.1,1,10],2,['Pay duly','delay > 1'],'Percentage','PAY_4_bins','% of population PAY_4','Default PAY_4 ')

group_biv_v1(credit.PAY_5,'PAY_5_range',[-2.1,1,10],2,['Pay duly','delay > 1'],'Percentage','PAY_5_bins','% of population PAY_5','Default PAY_5 ')

group_biv_v1(credit.PAY_6,'PAY_6_range',[-2.1,1,10],2,['Pay duly','delay > 1'],'Percentage','PAY_6_bins','% of population PAY_6','Default PAY_6 ')


In [89]:
credit.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT6,default_next_m,EDUCATION_INP,MARRIAGE_INP,PAY_1_range,PAY_2_range,PAY_3_range,PAY_4_range,PAY_5_range,PAY_6_range
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,1,2,1,delay > 1,delay > 1,Pay duly,Pay duly,Pay duly,Pay duly
1,2,120000,2,2,2,26,-1,2,0,0,...,2000,1,2,2,Pay duly,delay > 1,Pay duly,Pay duly,Pay duly,delay > 1
2,3,90000,2,2,2,34,0,0,0,0,...,5000,0,2,2,Pay duly,Pay duly,Pay duly,Pay duly,Pay duly,Pay duly
3,4,50000,2,2,1,37,0,0,0,0,...,1000,0,2,1,Pay duly,Pay duly,Pay duly,Pay duly,Pay duly,Pay duly
4,5,50000,1,2,1,57,-1,0,-1,0,...,679,0,2,1,Pay duly,Pay duly,Pay duly,Pay duly,Pay duly,Pay duly


In [90]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 33 columns):
ID                30000 non-null int64
LIMIT_BAL         30000 non-null int64
SEX               30000 non-null int64
EDUCATION         30000 non-null int64
MARRIAGE          30000 non-null int64
AGE               30000 non-null int64
PAY_1             30000 non-null int64
PAY_2             30000 non-null int64
PAY_3             30000 non-null int64
PAY_4             30000 non-null int64
PAY_5             30000 non-null int64
PAY_6             30000 non-null int64
BILL_AMT1         30000 non-null int64
BILL_AMT2         30000 non-null int64
BILL_AMT3         30000 non-null int64
BILL_AMT4         30000 non-null int64
BILL_AMT5         30000 non-null int64
BILL_AMT6         30000 non-null int64
PAY_AMT1          30000 non-null int64
PAY_AMT2          30000 non-null int64
PAY_AMT3          30000 non-null int64
PAY_AMT4          30000 non-null int64
PAY_AMT5          30000 non-null 

In [91]:
#### CREATE DUMMY VARIABLES
#Convert Categorical fields to bool using get_dummies
#Use drop_first=true to aviod multicolinierarity
### The drop_first=true eliminates the first group of the dummy variable
### According to the notes provided of the professor we do this to prevent multicollinearity
#### I can see it in the context of regression and logistic regression. We may want to select the group to delet
##### The method "get_dummies" also ELIMINATES THE ORIGINAL VARIABLE TO CREATE THE dummies

#EC: Creating  another reference in MEMORY, so the changes in df will not be REFLECTED IN CREDIT
df = credit.copy()



df = pd.get_dummies(data=credit, 
                       columns=['EDUCATION_INP',
                                'MARRIAGE_INP', 
                                'PAY_1_range',
                                'PAY_2_range',
                                'PAY_3_range',
                                'PAY_4_range',
                                'PAY_5_range',
                                'PAY_6_range',
                                'SEX'], drop_first=True) #Try drop_first=true to aviod multicolinierarity







df.info()

df.head()


#credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 37 columns):
ID                            30000 non-null int64
LIMIT_BAL                     30000 non-null int64
EDUCATION                     30000 non-null int64
MARRIAGE                      30000 non-null int64
AGE                           30000 non-null int64
PAY_1                         30000 non-null int64
PAY_2                         30000 non-null int64
PAY_3                         30000 non-null int64
PAY_4                         30000 non-null int64
PAY_5                         30000 non-null int64
PAY_6                         30000 non-null int64
BILL_AMT1                     30000 non-null int64
BILL_AMT2                     30000 non-null int64
BILL_AMT3                     30000 non-null int64
BILL_AMT4                     30000 non-null int64
BILL_AMT5                     30000 non-null int64
BILL_AMT6                     30000 non-null int64
PAY_AMT1                

,ID,LIMIT_BAL,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,MARRIAGE_INP_2,MARRIAGE_INP_3,PAY_1_range_delay 1 month,PAY_1_range_delay > 1,PAY_2_range_delay > 1,PAY_3_range_delay > 1,PAY_4_range_delay > 1,PAY_5_range_delay > 1,PAY_6_range_delay > 1,SEX_2
0,1,20000,2,1,24,2,2,-1,-1,-2,...,0,0,0,1,1,0,0,0,0,1
1,2,120000,2,2,26,-1,2,0,0,0,...,1,0,0,0,1,0,0,0,1,1
2,3,90000,2,2,34,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,4,50000,2,1,37,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,50000,2,1,57,-1,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,0


> Next, additional variables are created to help with the regression and classification analysis. The following variables were created and added to the dataset:
* Average bill amount over the 6 month period
* Average payment amount over the 6 month period
* Ratio of average bill amount to credit limit balance
* Ratio of the average payment amount to credit limit balance

In [92]:
##### CREATING ADDITIONAL VARIABLES 

############## CREATING NEW VARIABLES

#Average of the bill amount
df['Avg_BILL']=(df['BILL_AMT1']+df['BILL_AMT2']+df['BILL_AMT3']+ df['BILL_AMT4']+df['BILL_AMT5']+df['BILL_AMT6']
               )/float(6)


#Average of the Payment amount

df['Avg_PAY']=(df['PAY_AMT1']+df['PAY_AMT2']+df['PAY_AMT3']+df['PAY_AMT4']+df['PAY_AMT5']+df['PAY_AMT6']
               )/float(6)

#We are constructing the ratio with the average BILL AMOUNT TO LIMIT
df['AvgBill_to_LIMIT_BAL']=(df['Avg_BILL']/df['LIMIT_BAL'])


#We are constructing the ratio with the  average PAYMENT AMOUNT TO LIMIT
df['AvgPay_to_LIMIT_BAL']=(df['Avg_PAY']/df['LIMIT_BAL'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 41 columns):
ID                            30000 non-null int64
LIMIT_BAL                     30000 non-null int64
EDUCATION                     30000 non-null int64
MARRIAGE                      30000 non-null int64
AGE                           30000 non-null int64
PAY_1                         30000 non-null int64
PAY_2                         30000 non-null int64
PAY_3                         30000 non-null int64
PAY_4                         30000 non-null int64
PAY_5                         30000 non-null int64
PAY_6                         30000 non-null int64
BILL_AMT1                     30000 non-null int64
BILL_AMT2                     30000 non-null int64
BILL_AMT3                     30000 non-null int64
BILL_AMT4                     30000 non-null int64
BILL_AMT5                     30000 non-null int64
BILL_AMT6                     30000 non-null int64
PAY_AMT1                

### Removing variables not needed in the analysis
>We will conduct one classification task and one regression task. We will create 2 different objects since we will remove different variables

> We first create the object for the classification task

In [93]:
### WE CREATE AN OBJECT FOR THE CLASSIFICATION TASK AND OTHER FOR THE REGRESSION TASK

# CREATING THE OBJECT FOR THE CLASSIFICATION TASK
class_t=df.copy()


if 'default_next_m' in class_t:
    y_c = class_t['default_next_m'].values
    del class_t['default_next_m']
    del class_t['ID']
    del class_t['EDUCATION']
    del class_t['MARRIAGE']
    del class_t['PAY_1']
    del class_t['PAY_2']
    del class_t['PAY_3']
    del class_t['PAY_4']
    del class_t['PAY_5']
    del class_t['PAY_6']
    X_c = class_t.values
    
    
    

> Then we create the object for the Regression Task

In [94]:
### WE CREATE AN OBJECT FOR THE CLASSIFICATION TASK AND OTHER FOR THE REGRESSION TASK

# CREATING THE OBJECT FOR THE REGRESSION  TASK

reg_t=df.copy()


if 'AvgBill_to_LIMIT_BAL' in reg_t:
    y_r = reg_t['AvgBill_to_LIMIT_BAL'].values
    del reg_t['AvgBill_to_LIMIT_BAL']
    del reg_t['ID']
    del reg_t['EDUCATION']
    del reg_t['MARRIAGE']
    del reg_t['PAY_1']
    del reg_t['PAY_2']
    del reg_t['PAY_3']
    del reg_t['PAY_4']
    del reg_t['PAY_5']
    del reg_t['PAY_6']
    del reg_t['Avg_BILL']
    del reg_t['LIMIT_BAL']
    del reg_t['AvgPay_to_LIMIT_BAL']
    del reg_t['BILL_AMT1']
    del reg_t['BILL_AMT2']
    del reg_t['BILL_AMT3']
    del reg_t['BILL_AMT4']
    del reg_t['BILL_AMT5']
    del reg_t['BILL_AMT6']
    X_r = reg_t.values

#len(y_r)
y_r
#reg_t.info()

array([0.0642    , 0.02371806, 0.1882463 , ..., 0.39164444, 0.55543958,
       0.76958   ])

# <a name="Dataset-Description"></a>Dataset Description

>[5 points] Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

# <a name="Data-Meaning-Type"></a>Data Meaning Type
*Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file.*

> The original dataset contains the following data types:

| Variable | Variable Type | Description |
| :------: | :-----------: | :----- |
| LIMIT_BAL | int64 | The amount of the given credit in New Taiwan dollars.<br/>It includes both the individual consumer credit and his/her family (supplementary) credit |
| SEX | int64 | Defines each subjects gender with:<ul><li>1 = Male</li><li>2 = Female</li></ul> |
| EDUCATION | int64 | Represents each subject level of education with:<ul><li>1 = Graduate School</li><li>2 = University</li><li>3 = High School</li><li>4 = Other</li></ul> |
| MARRIAGE | int64 | Defines the person's relationship status by:<ul><li>1 = Married</li><li>2 = Single</li><li>3 = Other</li></ul> |
| AGE | int64 | Defines how old each person is in years |
| PAY_0 | int64 | The repayment status in September, 2005 |
| PAY_2 | int64 | The repayment status in August, 2005 |
| PAY_3 | int64 | The repayment status in July, 2005 |
| PAY_4 | int64 | The repayment status in June, 2005 |
| PAY_5 | int64 | The repayment status in May, 2005 |
| PAY_6 | int64 | The repayment status in April, 2005 |
| BILL_AMT1 | int64 | The repayment status in September, 2005 |
| BILL_AMT2 | int64 | The repayment status in August, 2005 |
| BILL_AMT3 | int64 | The repayment status in July, 2005 |
| BILL_AMT4 | int64 | The repayment status in June, 2005 |
| BILL_AMT5 | int64 | The repayment status in May, 2005 |
| BILL_AMT6 | int64 | The repayment status in April, 2005 |
| PAY_AMT1 | int64 | The amount paid in September, 2005 |
| PAY_AMT2 | int64 | The amount paid in August, 2005 |
| PAY_AMT3 | int64 | The amount paid in July, 2005 |
| PAY_AMT4 | int64 | The amount paid in June, 2005 |
| PAY_AMT5 | int64 | The amount paid in May, 2005 |
| PAY_AMT6 | int64 | The amount paid in April, 2005 |
| default payment next month | int64 | A binary value indicating default status with:<ul><li>0 = Customer has defaulted</li><li>1 = Customer has not defaulted</li></ul>|

### Dataset Description for the classification task

lab1 var for class

### Dataset Description for the Regression task

> To complete the regression task, several columns were removed. The remaining variables include age, payment amounts, default status for next month, the imputed values of education and marital status, delay in repayment status, sex, and average payment amount. The four new created variables, as mentioned above, were added to the regression dataset as well.

# <a name="Evaluation-Metrics-Description"></a>Evaluation Metrics Description


>[10 points] Choose and explain your evaluation metrics that you will use (i.e., accuracy,
precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

roc recall, etc.

# <a name="Training-Testing"></a>Training and Testing Splits method

>[10 points] Choose the method you will use for dividing your data into training and testing splits 
(i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is 
appropriate or use more than one method as appropriate. For example, 
if you are using time series data then you should be using continuous training and testing sets across time.


for classisifations: We chose to use the stratified sampling method becuase our data contains a significantly high number of subjects classified as “0”. Because of this inbalance, there is potential for the algorithm to predict a high number of false negatives and flase  becaue of the class imbalance. The stratified cross validation procedure will ensure the training data sets will have approximately the same amount of subjects who defaulted as those who did not and will subsequently protect the algorithm against false negative predictions

for regeression: 

In [107]:
#### DEFINING THE CV OBJECT FOR CLASSIFICATION

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

num_cv_iterations = 10

cv_object = StratifiedShuffleSplit(n_splits = num_cv_iterations, 
                            test_size = 0.20, train_size = 0.80, random_state=1)


##### We generate the training and testing sample
for train_indices, test_indices in cv_object.split(X_c,y_c): 
    X_c_train = X_c[train_indices]
    y_c_train = y_c[train_indices]
    
    X_c_test = X_c[test_indices]
    y_c_test = y_c[test_indices]


#### SCALING THE OBJECTS
    


# we want to normalize the features based upon the mean and standard deviation of each column. 
# However, we do not want to accidentally use the testing data to find out the mean and std (this would be snooping)
# to Make things easier, let's start by just using whatever was last stored in the variables:
##    X_train , y_train , X_test, y_test (they were set in a for loop above)

# scale attributes by the training set
scl_obj = StandardScaler()
scl_obj.fit(X_c_train) # find scalings for each column that make this zero mean and unit std
# the line of code above only looks at training data to get mean and std and we can use it 
# to transform new feature data

X_c_train_scaled = scl_obj.transform(X_c_train) # apply to training
X_c_test_scaled = scl_obj.transform(X_c_test) # apply those means and std to the test set (without snooping at the test set values)
    
    
    
    
    



In [100]:
#### DEFINING THE CV OBJECT FOR REGRESSION

from sklearn.model_selection import ShuffleSplit
#cv = ShuffleSplit(n_splits=10, test_size=0.10, random_state=0)
#from sklearn.model_selection import StratifiedShuffleSplit

num_cv_iterations = 10

cv_reg = ShuffleSplit(n_splits = num_cv_iterations, 
                            test_size = 0.20, train_size = 0.80, random_state=1)


##### We generate the training and testing sample
for train_indices, test_indices in cv_reg.split(X_r,y_r): 
    X_r_train = X_r[train_indices]
    y_r_train = y_r[train_indices]
    
    X_r_test = X_r[test_indices]
    y_r_test = y_r[test_indices]


#cv_reg

# <a name="Classification-Task"></a>Classification Task

> [20 points] Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!

## Random Forest Classification

In [2]:
# Source:
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=4,
                            n_informative=2, n_redundant=0,
                            random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=4, random_state=0)
clf.fit(X, y)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
print(clf.feature_importances_)
print(clf.predict([[0, 0, 0, 0]]))

[ 0.09455901  0.84576276  0.02488277  0.03479546]
[1]


## KNN NEIGHBORS Classification

## Logistic Regression Classification

# <a name="Regression-Task"></a>Regression Task

>[20 points] Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!

## Custom Scorers for Evaluating Regression Models 

**All regression models created in this notebook are validated using the following metrics:**
* Mean Absolute Error (MAE)
* Root Mean Squared Error (RMSE) - https://stackoverflow.com/questions/17197492/root-mean-square-error-in-python
* Mean Absolute Percentage Error (MAPE) - https://stackoverflow.com/questions/42250958/how-to-optimize-mape-code-in-python

**For details on making scorers to return multiple mean error scores see:**
* http://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html
* https://github.com/scikit-learn/scikit-learn/pull/7388
* https://github.com/drorata/multiscorer

In [101]:
##### SETTING UP THE PERFORMANCE METRICS
#Use mean absolute error (MAE) to score the regression models created 
#(the scale of MAE is identical to the response variable)
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error

#Function for Root mean squared error
#https://stackoverflow.com/questions/17197492/root-mean-square-error-in-python
def rmse(y_actual, y_predicted):
    return np.sqrt(mean_squared_error(y_actual, y_predicted))

#Function for Mean Absolute Percentage Error (MAPE) - Untested
#Adapted from - https://stackoverflow.com/questions/42250958/how-to-optimize-mape-code-in-python
def mape(y_actual, y_predicted): 
    mask = y_actual != 0
    return (np.fabs(y_actual - y_predicted)/y_actual)[mask].mean() * 100

#Create scorers for rmse and mape functions
mae_scorer = make_scorer(score_func=mean_absolute_error, greater_is_better=False)
rmse_scorer = make_scorer(score_func=rmse, greater_is_better=False)
mape_scorer = make_scorer(score_func=mape, greater_is_better=False)

#Make scorer array to pass into cross_validate() function for producing mutiple scores for each cv fold.
errorScoring = {'MAE':  mae_scorer, 
                'RMSE': rmse_scorer,
                'MAPE': mape_scorer
               } 

## Regression Model Evaluation
** All regression models are evaluated using the regression model evaluation function below: ** 
* The following regression evaluation function uses the cross validation object and the custom scorers in the two cells above in combination with sklearn.model_selection's cross_validate function to perform cross validation for regression estimators.
* The cross validation object above uses a random seed to ensure that all regression estimators are tested on the same randomly selected records for each cross validation fold.
* Custom scorers are created using the three chosen mean error scores and passed into cross_validate(), so all three scores are calcualted using a single call to cross_validate().
* All of this functionality is wrapped within the custom EvaluateRegressionEstimator() function below so multiple regression models may be tested using the same test / train cv data and evaluation scores producing a consistent output for each model without the need to re-write the same code over and over. 

In [102]:
from sklearn.model_selection import cross_validate

def EvaluateRegressionEstimator(regEstimator, X, y, cv):
    
    scores = cross_validate(regEstimator, X, y, scoring=errorScoring, cv=cv, return_train_score=True)

    #cross val score sign-flips the outputs of MAE
    # https://github.com/scikit-learn/scikit-learn/issues/2439
    scores['test_MAE'] = scores['test_MAE'] * -1
    scores['test_MAPE'] = scores['test_MAPE'] * -1
    scores['test_RMSE'] = scores['test_RMSE'] * -1

    #print mean MAE for all folds 
    maeAvg = scores['test_MAE'].mean()
    print_str = "The average MAE for all cv folds is: \t\t\t {maeAvg:.5}"
    print(print_str.format(maeAvg=maeAvg))

    #print mean test_MAPE for all folds
    scores['test_MAPE'] = scores['test_MAPE']
    mape_avg = scores['test_MAPE'].mean()
    print_str = "The average MAE percentage (MAPE) for all cv folds is: \t {mape_avg:.5}"
    print(print_str.format(mape_avg=mape_avg))

    #print mean MAE for all folds 
    RMSEavg = scores['test_RMSE'].mean()
    print_str = "The average RMSE for all cv folds is: \t\t\t {RMSEavg:.5}"
    print(print_str.format(RMSEavg=RMSEavg))
    print('*********************************************************')

    print('Cross Validation Fold Mean Error Scores')
    scoresResults = pd.DataFrame()
    scoresResults['MAE'] = scores['test_MAE']
    scoresResults['MAPE'] = scores['test_MAPE']
    scoresResults['RMSE'] = scores['test_RMSE']
    return scoresResults

## Baseline Regression Model

**Linear Regression is used to create a baseline model.  Since linear regression may predict response variable values outside the range of the training data's response variable, we create a linear regression estimator with minority percentage predictions clipped 0% and 100%. For details see:**
* http://scikit-learn.org/stable/developers/contributing.html#rolling-your-own-estimator 
* https://github.com/scikit-learn/scikit-learn/issues/6950
* http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
* https://github.com/scikit-learn-contrib/project-template/blob/master/skltemplate/template.py
* https://stackoverflow.com/questions/44234682/how-to-use-sklearn-when-target-variable-is-a-proportion

In [103]:
#Make new estimator compatible for use with GridSearchCV() and cross_validate()
# -  Cap predict function for LinearRegression between 0 and 100
# -  See: Roll your own estimator links above for details. 
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.linear_model import LinearRegression

class CappedLinearRegression(LinearRegression):

    def predict(self, X):
        return np.clip(super(CappedLinearRegression, self).predict(X), 0, 100) 

### Linear Regression Grid Search
** Here we perform a grid search testing 40 models to find the best parameters for our Linear Regression model based on Mean Absolute Error.  See more on parameter tuning with grid search here:**
* http://scikit-learn.org/stable/modules/grid_search.html

In [104]:
#Create a Linear Regression object and perform a grid search to find the best parameters
linreg = CappedLinearRegression()
parameters = {'normalize':(True,False), 'fit_intercept':(True,False)}

#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=linreg
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv_reg # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
#regGridSearch.fit(X_r, y_r)
regGridSearch.fit(X_r_train, y_r_train)

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.4s finished


GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=1, test_size=0.2, train_size=0.8),
       error_score='raise',
       estimator=CappedLinearRegression(copy_X=True, fit_intercept=True, n_jobs=1,
            normalize=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'normalize': (True, False), 'fit_intercept': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(mean_absolute_error, greater_is_better=False),
       verbose=1)

In [105]:
#Print the parameterization of the best estimator
regGridSearch.best_estimator_

CappedLinearRegression(copy_X=True, fit_intercept=True, n_jobs=1,
            normalize=False)

##  Baseline Regression Model - Cross Validation
**Perform tenfold cross validation using the grid search "best" parameters and our Capped Linear Regression estimator**
* 10-fold cross-validation using the parameters for the top performing model 
* CAP predictions between 0 and 100% 
* Evaluate cross-validation results using MAE, MAPE, and RMSE

In [106]:
#Create CappedLinearRegression predictions between 0 and 100% using the best parameters for our Linear Regression object
regEstimator = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics. 
#EvaluateRegressionEstimator(regEstimator, X_r, y_r, cv_reg)
EvaluateRegressionEstimator(regEstimator, X_r_train, y_r_train, cv_reg)


The average MAE for all cv folds is: 			 0.27763
The average MAE percentage (MAPE) for all cv folds is: 	 625.39
The average RMSE for all cv folds is: 			 0.32532
*********************************************************
Cross Validation Fold Mean Error Scores


,MAE,MAPE,RMSE
0,0.278346,-1537.906701,0.324814
1,0.279350,1921.055181,0.327116
2,0.273167,-1642.538180,0.318002
3,0.277416,-1493.481073,0.327031
4,0.276687,1858.570440,0.321749
5,0.279477,1966.131017,0.329058
6,0.275913,1655.264369,0.326384
7,0.280647,1430.388675,0.329018
8,0.278445,912.709904,0.325872
9,0.276810,1183.682340,0.324132


##  Lasso Regression
**Perform regression using Linear Model trained with L1 prior as regularizer (aka the Lasso)**

Documentation: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [99]:
#Create a regression object and perform a grid search to find the best parameters
from sklearn.linear_model import Lasso

lassoreg = Lasso(fit_intercept=True, normalize=True,copy_X=True
          , max_iter=10000, precompute=True, tol=0.0001, random_state=0)

#Test parameters 
alpha = [0.001, 0.1, 1, 10, 20]
selection = ['cyclic','random']
warm_start = [True, False]
parameters = {'alpha': alpha, 'selection': selection, 'warm_start': warm_start}

#Create a grid search object using the parameters above
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=lassoreg
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv_reg # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X_r, y_r)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    4.9s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    9.4s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    9.7s finished


GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=1, test_size=0.2, train_size=0.8),
       error_score='raise',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=10000,
   normalize=True, positive=False, precompute=True, random_state=0,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'alpha': [0.001, 0.1, 1, 10, 20], 'warm_start': [True, False], 'selection': ['cyclic', 'random']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(mean_absolute_error, greater_is_better=False),
       verbose=1)

In [100]:
#Display the best estimator parameters
regGridSearch.best_estimator_

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=10000,
   normalize=True, positive=False, precompute=True, random_state=0,
   selection='cyclic', tol=0.0001, warm_start=True)

In [101]:
#Create a regression estimator with best parameters for cross validation
regEstimator = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics.
EvaluateRegressionEstimator(regEstimator, X_r, y_r, cv_reg)

The average MAE for all cv folds is: 			 0.31171
The average MAE percentage (MAPE) for all cv folds is: 	 -379.45
The average RMSE for all cv folds is: 			 0.353
*********************************************************
Cross Validation Fold Mean Error Scores


,MAE,MAPE,RMSE
0,0.312945,1968.562203,0.357046
1,0.310933,-557.090559,0.349272
2,0.310459,-2133.992640,0.350454
3,0.309206,821.386470,0.346295
4,0.312353,-848.455682,0.352686
5,0.307881,-2452.067162,0.348565
6,0.313857,-1264.995682,0.355773
7,0.315162,548.224661,0.361312
8,0.311389,-1114.752323,0.350986
9,0.312897,1238.684465,0.357586


In [102]:
#Do we predict Average Bill amount to card Limit Balance ratios greater than 100%?
regEstimator = Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=10000,
   normalize=True, positive=False, precompute=True, random_state=0,
   selection='cyclic', tol=0.0001, warm_start=True)

regEstimator.fit(X_r, y_r)
yhat = regEstimator.predict(X_r)
print("Yhat Max: ", yhat.max())

Yhat Max:  0.373047945753


##  Ridge Regression
**Perform regression using Linear least squares with l2 regularization**

Documentation: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [103]:
#Create a regression object and perform a grid search to find the best parameters
from sklearn.linear_model import Ridge

ridgereg = Ridge(fit_intercept=True, normalize=True,copy_X=True
          , max_iter=1000, tol=0.0001, random_state=0)

#Test parameters 
alpha = [0.001, 0.1, 1, 5, 10, 20]
solver = [ 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
parameters = {'alpha': alpha, 'solver': solver}

#Create a grid search object using the parameters above
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=ridgereg
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv_reg # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X_r, y_r)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    4.3s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   19.0s
[Parallel(n_jobs=8)]: Done 360 out of 360 | elapsed:   30.8s finished


GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=1, test_size=0.2, train_size=0.8),
       error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, random_state=0, solver='auto', tol=0.0001),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'], 'alpha': [0.001, 0.1, 1, 5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(mean_absolute_error, greater_is_better=False),
       verbose=1)

In [104]:
#Display the best estimator parameters
regGridSearch.best_estimator_

Ridge(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, random_state=0, solver='lsqr', tol=0.0001)

In [105]:
#Create a regression estimator with best parameters for cross validation
regEstimator = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics.
EvaluateRegressionEstimator(regEstimator, X_r, y_r, cv_reg)

The average MAE for all cv folds is: 			 0.27825
The average MAE percentage (MAPE) for all cv folds is: 	 351.59
The average RMSE for all cv folds is: 			 0.32815
*********************************************************
Cross Validation Fold Mean Error Scores


,MAE,MAPE,RMSE
0,0.280175,1662.023074,0.332624
1,0.276132,-539.584402,0.321971
2,0.278355,-1255.239839,0.326401
3,0.276066,1338.978267,0.322782
4,0.277515,810.626879,0.324810
5,0.275993,-233.839916,0.324786
6,0.280331,-612.434324,0.330710
7,0.279359,442.836824,0.336359
8,0.278508,641.759902,0.326381
9,0.280114,1260.757413,0.334655


In [106]:
#Do we predict Average Bill amount to card Limit Balance ratios greater than 100%?
regEstimator = Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, random_state=0, solver='saga', tol=0.0001)

regEstimator.fit(X_r, y_r)
yhat = regEstimator.predict(X_r)
print("Yhat Max: ", yhat.max())

Yhat Max:  2.21058516598


# <a name="Methods-of-Evaluation"></a>Methods of Evaluation Results

> [10 points] Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.




classification: ROC, acc, recall. Regression: mse, acc, rmse, mae mape

# <a name="Advantages-of-each-model"></a>Advantages of each model
>[10 points] Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the cours


all add your own part. we can clean up after all ideas are put down. 

# <a name="Relevant-Attributes"></a>Relevant Attributes
>[10 points] Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task


all add your own part. we can clean up after all ideas are put down. 

# <a name="Deployment"></a>Deployment
>[5 points] How useful is your model for interested parties (i.e., the companies or organizations
that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?

all add your own part. we can clean up after all ideas are put down. 

# <a name="Exceptional-Work"></a>Exceptional Work
> [10 points] You have free reign to provide additional analyses
> One idea: grid search parameters in a parallelized fashion and visualize the
performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?

grid search- ernesto

# <a name="Training-and-Testing-Split"></a>Training and Testing Split

## Stratified K-Folds Cross-Validation

> There is a very apparent class imbalance within the dataset. In order to address this issue, stratified K-Folds cross-validation will be employed to preserve the percentage of samples for each class. In other words, this will prevent the model from simply guessing 'paid duly' for each observation.

In [ ]:
#df = credit
#EC: Creating  another reference in MEMORY, so the changes in df will not be REFLECTED IN CREDIT
#df = credit.copy()
from sklearn.model_selection import StratifiedShuffleSplit
if 'default_next_m' in df:
    y_K = df['default_next_m'].values
    del df['default_next_m']
    del df['ID']
    del df['PAY_1']
    del df['PAY_2']
    del df['PAY_3']
    del df['PAY_4']
    del df['PAY_5']
    del df['PAY_6']
    del df['Avg_BILL']
    del df['LIMIT_BAL']
    del df['AvgPay_to_LIMIT_BAL']
    del df['BILL_AMT1']
    del df['BILL_AMT2']
    del df['BILL_AMT3']
    del df['BILL_AMT4']
    del df['BILL_AMT5']
    del df['BILL_AMT6']
    X_K = df.values
num_cv_iterations = 3
num_instances = len(y)
cv_object = StratifiedShuffleSplit(n_splits = num_cv_iterations, 
                            test_size = 0.20, train_size = 0.80, random_state=1)
cv_object.get_n_splits(X, y)
print(cv_object)
for train_index, test_index in cv_object.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
df.head()

In [ ]:
print('# of obs in train set: ' + str(len(y_train)))
print('# of obs in test set: ' + str(len(y_test)))

#train_defaults = sum(y_train == 0)
train_defaults = sum(y_train == 1)
print('# of defaults in train set: ' + str(train_defaults))

proportion_train_defaults = train_defaults / len(y_train)
print('Proportion of defaults in train set: ' + str(proportion_train_defaults))

#test_defaults = sum(y_test == 0)
test_defaults = sum(y_test == 1)
print('# of defaults in test set: ' + str(test_defaults))

proportion_test_defaults = test_defaults / len(y_test)
print('Proportion of defaults in test set: ' + str(proportion_test_defaults))

In [ ]:
print('# of obs in train set: ' + str(len(y_train)))
print('# of obs in test set: ' + str(len(y_test)))

#train_defaults = sum(y_train == 0)
train_defaults = sum(y_train == 1)
print('# of defaults in train set: ' + str(train_defaults))

proportion_train_defaults = train_defaults / len(y_train)
print('Proportion of defaults in train set: ' + str(proportion_train_defaults))

#test_defaults = sum(y_test == 0)
test_defaults = sum(y_test == 1)
print('# of defaults in test set: ' + str(test_defaults))

proportion_test_defaults = test_defaults / len(y_test)
print('Proportion of defaults in test set: ' + str(proportion_test_defaults))

In [ ]:
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.html import widgets 
from sklearn.metrics import accuracy_score 
from sklearn import metrics as mt


max_accs = 0
max_k = 0

for k in range(5,15):
    clf = KNeighborsClassifier(n_neighbors=k, weights='uniform', metric='cosine')
    clf.fit(X_train, y_train)
    yhat = clf.predict(X_test)
    acc = mt.accuracy_score(y_test,yhat)
    if max(max_accs, acc) == acc:
        max_accs = acc
        max_k = k
    print('Accuracy of', k, 'neighbors is', acc * 100, '%')

print('\nThe best accuracy is %f with %d neighbor(s)'%(max_accs,max_k))

In [ ]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

for d in ['l1','l2','cosine']:
    clf = NearestCentroid(metric=d)
    clf.fit(X_train, y_train)
    yhat = clf.predict(X_test)
    acc = mt.accuracy_score(y_test,yhat)
    print('\nAccuracy of', d, 'metrics is', acc * 100, '%')    
    print('The best distance metric is: ', d)

In [ ]:
lr_clf = LogisticRegression(penalty='l2', C=1, class_weight='balanced') # get object
#WARNING: THE FIRST WIEGHT WAS 1 
iter_num=0
times_rec=[]
for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
     #we count the time in executing the logistic regression
    t0 = time()
    
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions
    
    t1=time()
    diff=np.array([t1-t0])
    
    print ("The time it takes to fit and predict is " + str(diff[0]) + "\n")    
    times_rec=np.append(times_rec,diff)
    

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    
    print("\n *** CLASSIFICATION REPORT ****")
    #### CLASSIFICATION REPORT
    ClassReport = mt.classification_report(y_test,y_hat)
    print(ClassReport)
    
    iter_num+=1
    
    
print("The average time to fit and predict 3 logistic regressions with 80/20 training/test split is: " + str(times_rec.mean()) )

# <a name="References"></a>References

* https://github.com/eclarson/DataMiningNotebooks/blob/master/04.%20Logits%20and%20SVM.ipynb
* https://github.com/jakemdrew/EducationDataNC/blob/master/Graduation%20Rates%20v2.ipynb
* http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation
* http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
* https://stackoverflow.com/questions/29438265/stratified-train-test-split-in-scikit-learn
* http://blog.exsilio.com/all/accuracy-precision-recall-f1-score-interpretation-of-performance-measures/
* https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/
* http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
* http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html
* http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

